# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet'])
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.multiclass import type_of_target
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from scipy import stats
import sys
import pickle

[nltk_data] Downloading package punkt to /Users/AlinaV/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/AlinaV/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Database.db')
df = pd.read_sql_table('Table', engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
Y = df[df.columns[4:]]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def display_results(y_test, y_pred):

    for i, column in enumerate(y_test):
        clf_report = classification_report(y_test[column], y_pred[:, i], zero_division=0)
        print(column, ":", clf_report)
        

In [9]:
display_results(y_test, y_pred)

related :               precision    recall  f1-score   support

           0       0.76      0.26      0.39      1540
           1       0.81      0.97      0.88      4967

    accuracy                           0.81      6507
   macro avg       0.79      0.62      0.64      6507
weighted avg       0.80      0.81      0.77      6507

request :               precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.88      0.43      0.58      1062

    accuracy                           0.90      6507
   macro avg       0.89      0.71      0.76      6507
weighted avg       0.90      0.90      0.88      6507

offer :               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99   

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
from sklearn.model_selection import GridSearchCV

parameters = { 
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200]
}
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
display_results(y_test, y_pred)
print("\nBest Parameters:", cv.best_params_)

related :               precision    recall  f1-score   support

           0       0.78      0.27      0.40      1540
           1       0.81      0.98      0.89      4967

    accuracy                           0.81      6507
   macro avg       0.79      0.62      0.64      6507
weighted avg       0.80      0.81      0.77      6507

request :               precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.89      0.44      0.59      1062

    accuracy                           0.90      6507
   macro avg       0.89      0.71      0.76      6507
weighted avg       0.90      0.90      0.88      6507

offer :               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
clf_params = list(cv.best_params_.values())[0]
tfidf_params = list(cv.best_params_.values())[1]

In [25]:
# use best results to build the final model
model = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer(use_idf=tfidf_params)), 
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=clf_params)))
])
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
display_results(y_test, y_pred)

related :               precision    recall  f1-score   support

           0       0.76      0.27      0.40      1540
           1       0.81      0.97      0.89      4967

    accuracy                           0.81      6507
   macro avg       0.79      0.62      0.64      6507
weighted avg       0.80      0.81      0.77      6507

request :               precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.89      0.42      0.57      1062

    accuracy                           0.90      6507
   macro avg       0.89      0.70      0.75      6507
weighted avg       0.90      0.90      0.88      6507

offer :               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99   

### 9. Export your model as a pickle file

In [27]:
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.